# EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
csvfile = "meetings_complete.csv"

In [3]:
df = pd.read_csv(csvfile)

In [4]:
df.head()

,Name,Day,Time,Video,Desc,Categories,Email,Phone
0,1 Hole in the Sky AA Meetings: AA Secular Dail...,Tuesday,5:00 pm,https://zoom.us/j/414948190,Tuesday: AA Secular Daily Reflections 3:00pm P...,"Audio,Open,Video",NaN,NaN
1,AA Friends,Tuesday,5:00 pm,https://aafriends.webs.com/,AA Friends currently holds three online meetin...,"Forum,Open",aafriends@usa.com,NaN
2,AA New Beginnings,Tuesday,5:00 pm,https://us02web.zoom.us/j/5662326551,"A daily meeting that focuses on experience, st...","Audio,Big Book,Discussion,Open,Video",AANewBeginnings230@gmail.com,NaN
3,Agnes Water,Tuesday,5:00 pm,https://us02web.zoom.us/j/8983968244,PW billw,"Big Book,Discussion,Speaker,Video",agneswaterxa@gmail.com,NaN
4,Beginners AA meeting,Tuesday,5:00 pm,https://us02web.zoom.us/j/88198562544?pwd=WFZm...,"Zoom Passcode, if any: serenity","Big Book,Discussion,Open,Speaker,Video",liaazoom2020@gmail.com,NaN


### Transforming

##### Convert 'Time' column to datetime data type

In [5]:
pd.to_datetime(df["Time"]).dt.time

0       17:00:00
1       17:00:00
2       17:00:00
3       17:00:00
4       17:00:00
          ...   
4162         NaT
4163         NaT
4164         NaT
4165         NaT
4166         NaT
Name: Time, Length: 4167, dtype: object

In [6]:
# dt.normalize() sets time component to 00:00:00, so here we're subtracting the days
pd.to_datetime(df["Time"]) - pd.to_datetime(df["Time"]).dt.normalize()

0      17:00:00
1      17:00:00
2      17:00:00
3      17:00:00
4      17:00:00
         ...   
4162        NaT
4163        NaT
4164        NaT
4165        NaT
4166        NaT
Name: Time, Length: 4167, dtype: timedelta64[ns]

In this case, it may be that the time data would be better analyzed as categorical. But since there is an ordinal component to it ("how many of this earlier in the day or that later in the day?"), let's just save both versions.

In [7]:
df["Time_dt"] = pd.to_datetime(df["Time"]).dt.time
df["Time_del"] = pd.to_datetime(df["Time"]) - pd.to_datetime(df["Time"]).dt.normalize()
df.head()

,Name,Day,Time,Video,Desc,Categories,Email,Phone,Time_dt,Time_del
0,1 Hole in the Sky AA Meetings: AA Secular Dail...,Tuesday,5:00 pm,https://zoom.us/j/414948190,Tuesday: AA Secular Daily Reflections 3:00pm P...,"Audio,Open,Video",NaN,NaN,17:00:00,17:00:00
1,AA Friends,Tuesday,5:00 pm,https://aafriends.webs.com/,AA Friends currently holds three online meetin...,"Forum,Open",aafriends@usa.com,NaN,17:00:00,17:00:00
2,AA New Beginnings,Tuesday,5:00 pm,https://us02web.zoom.us/j/5662326551,"A daily meeting that focuses on experience, st...","Audio,Big Book,Discussion,Open,Video",AANewBeginnings230@gmail.com,NaN,17:00:00,17:00:00
3,Agnes Water,Tuesday,5:00 pm,https://us02web.zoom.us/j/8983968244,PW billw,"Big Book,Discussion,Speaker,Video",agneswaterxa@gmail.com,NaN,17:00:00,17:00:00
4,Beginners AA meeting,Tuesday,5:00 pm,https://us02web.zoom.us/j/88198562544?pwd=WFZm...,"Zoom Passcode, if any: serenity","Big Book,Discussion,Open,Speaker,Video",liaazoom2020@gmail.com,NaN,17:00:00,17:00:00


##### Weekdays are ordered categorical data

In [8]:
from pandas.api.types import CategoricalDtype
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Ongoing']
day_type = CategoricalDtype(categories=days, ordered=True)
df['Day'] = df['Day'].astype(day_type)

##### Create new columns from category labels

In [9]:
all_labels = []
for field in df['Categories']:
    all_labels.extend(field.split(','))
labels = pd.unique(all_labels)
labels

array(['Audio', 'Open', 'Video', 'Forum', 'Big Book', 'Discussion',
       'Speaker', 'Closed', 'Telephone', 'Women',
       'Loners / Isolationists', 'Chat', 'LGBTQIAA+', 'Young People',
       'Men', 'Secular', 'French', 'Steps / Traditions',
       'ASL Interpreter', 'Game', 'Blind / Visually Impaired',
       'Deaf / Hard of Hearing', 'Email', 'Professionals',
       'Openm Discussion. Women', 'Meditation', 'Swedish', 'Seniors',
       'ASL', 'Newcomer', 'Hindi', 'Punjabi', '24/7'], dtype=object)

**What is this 'Openm Discussion. Women' value?**

In [10]:
df.loc[df['Categories'].str.contains('Openm Discussion. Women')]

,Name,Day,Time,Video,Desc,Categories,Email,Phone,Time_dt,Time_del
240,WHAM-Women Having A Meeting,Tuesday,9:00 pm,https://us02web.zoom.us/j/81398782806,"Other Access Code, if any Passcode: Wom3n","Openm Discussion. Women,Telephone,Video",WHAM.AA@aol.com,16699006833,21:00:00,21:00:00


Looking at the other category labels, we can go out on a limb and assume this one record is supposed to be Open, Discussion, and Women.

In [11]:
df.at[240, 'Categories'] = 'Open,Discussion,Women,Telephone,Video'
df.loc[240]

Name                        WHAM-Women Having A Meeting
Day                                             Tuesday
Time                                            9:00 pm
Video             https://us02web.zoom.us/j/81398782806
Desc          Other Access Code, if any Passcode: Wom3n
Categories        Open,Discussion,Women,Telephone,Video
Email                                   WHAM.AA@aol.com
Phone                                       16699006833
Time_dt                                        21:00:00
Time_del                                0 days 21:00:00
Name: 240, dtype: object

Much better. Hold on, this is supposed to consist of only English-language meetings. What about the other language labels?

In [12]:
other_langs = ['French', 'Swedish', 'Hindi', 'Punjabi']
df.loc[df['Categories'].str.contains('|'.join(other_langs))]

,Name,Day,Time,Video,Desc,Categories,Email,Phone,Time_dt,Time_del
85,Vivre et Laisser Vivre,Tuesday,6:30 pm,https://zoom.us/j/497838842,"+16699006833,,497838842# US (San Jose)","Audio,French,Open,Telephone,Video",NaN,"16465588656,,497838842",18:30:00,18:30:00
347,Swenglish Lunch Meeting,Wednesday,6:00 am,https://zoom.us/j/4554487458,Web Site: https://zoom.us/j/4554487458,Swedish,NaN,NaN,06:00:00,06:00:00
969,Swenglish Lunch Meeting,Thursday,6:00 am,https://zoom.us/j/4554487458,Web Site: https://zoom.us/j/4554487458,Swedish,NaN,NaN,06:00:00,06:00:00
1334,South Asian Group of USA,Thursday,7:00 pm,https://us02web.zoom.us/j/81322055835,Room opens at 7.45 PM and stays Open for Fello...,"Audio,Discussion,Hindi,Open,Speaker,Telephone,...",SouthAsianGroupUSA@gmail.com,6699006833,19:00:00,19:00:00
1552,Swenglish Lunch Meeting,Friday,6:00 am,https://zoom.us/j/4554487458,Web Site: https://zoom.us/j/4554487458,Swedish,NaN,NaN,06:00:00,06:00:00
2135,Swenglish Lunch Meeting,Saturday,6:00 am,https://zoom.us/j/4554487458,Web Site: https://zoom.us/j/4554487458,Swedish,NaN,NaN,06:00:00,06:00:00
2688,Swenglish Lunch Meeting,Sunday,6:00 am,https://zoom.us/j/4554487458,Web Site: https://zoom.us/j/4554487458,Swedish,NaN,NaN,06:00:00,06:00:00
2870,The Serenity Group (Sinnesrogruppen) in Sweden...,Sunday,12:30 pm,https://us02web.zoom.us/j/4040899397,Shares can be in English or Swedish.,"Open,Speaker,Swedish,Video",sinnesro.stockholm@gmail.com,NaN,12:30:00,12:30:00
3243,Swenglish Lunch Meeting,Monday,6:00 am,https://zoom.us/j/4554487458,Web Site: https://zoom.us/j/4554487458,Swedish,NaN,NaN,06:00:00,06:00:00
3468,Monday Punjabi/English Speaker Meeting,Monday,3:00 pm,https://us02web.zoom.us/j/35544906,ਹਰ ਸੋਮਵਾਰ ਨੂੰ GMT ਵਕ਼ਤ ਦੇ ਹਿਸਾਬ ਨਾਲ ਸ਼ਾਮ ਦੇ 8...,"Discussion,Open,Punjabi,Speaker,Video",nomorepretending@outlook.com,NaN,15:00:00,15:00:00


While only the Punjabi meeting has a non-English description, half of these only have a website for a description, so who really knows. These rows make up such a small percentage that I am inclined to simply remove them.

In [13]:
nonenglish = df[df['Categories'].str.contains('|'.join(other_langs))]
df = df.drop(nonenglish.index).reset_index(drop=True)

**Back to making dummies...**

In [14]:
all_labels = []
for field in df['Categories']:
    all_labels.extend(field.split(','))
labels = pd.unique(all_labels)
labels

array(['Audio', 'Open', 'Video', 'Forum', 'Big Book', 'Discussion',
       'Speaker', 'Closed', 'Telephone', 'Women',
       'Loners / Isolationists', 'Chat', 'LGBTQIAA+', 'Young People',
       'Men', 'Secular', 'Steps / Traditions', 'ASL Interpreter', 'Game',
       'Blind / Visually Impaired', 'Deaf / Hard of Hearing', 'Email',
       'Professionals', 'Meditation', 'Seniors', 'ASL', 'Newcomer',
       '24/7'], dtype=object)

In [15]:
zero_matrix = np.zeros((len(df), len(labels)))
dummies = pd.DataFrame(zero_matrix, columns=labels)

In [16]:
for i, labels in enumerate(df['Categories']):
    indices = dummies.columns.get_indexer(labels.split(','))
    dummies.iloc[i, indices] = 1

In [17]:
dummies.head()

,Audio,Open,Video,Forum,Big Book,Discussion,Speaker,Closed,Telephone,Women,...,Game,Blind / Visually Impaired,Deaf / Hard of Hearing,Email,Professionals,Meditation,Seniors,ASL,Newcomer,24/7
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df = df.join(dummies.add_prefix('Label_'))
df.iloc[0]

Name                               1 Hole in the Sky AA Meetings: AA Secular Dail...
Day                                                                          Tuesday
Time                                                                         5:00 pm
Video                                                    https://zoom.us/j/414948190
Desc                               Tuesday: AA Secular Daily Reflections 3:00pm P...
Categories                                                          Audio,Open,Video
Email                                                                            NaN
Phone                                                                            NaN
Time_dt                                                                     17:00:00
Time_del                                                             0 days 17:00:00
Label_Audio                                                                        1
Label_Open                                                       

##### Add column to indicate Zoom video links

Zoom has become far and wide the most popular video meeting application in society at large. Let's add one more dummy variable to capture video link URLs that contain the string 'zoom'.

In [19]:
df["Video"] = df["Video"].str.lower()
df["Zoom"] = df["Video"].str.contains('zoom')
df["Zoom"]

0        True
1       False
2        True
3        True
4        True
        ...  
4150      NaN
4151    False
4152    False
4153    False
4154      NaN
Name: Zoom, Length: 4155, dtype: object

In [20]:
df["Zoom"].value_counts()

True     3317
False     538
Name: Zoom, dtype: int64

### Describing

In [21]:
df.shape

(4155, 39)

In [22]:
df["Time_del"].min(), df["Time_del"].max(), df["Time_del"].mean(), df["Time_del"].mode()

(Timedelta('0 days 00:00:00'),
 Timedelta('0 days 23:59:00'),
 Timedelta('0 days 14:10:38.357037'),
 0   19:00:00
 dtype: timedelta64[ns])

In [23]:
# Remember these outliers & make histogram
df['Time_del'].value_counts()

19:00:00    327
11:00:00    278
20:00:00    255
18:00:00    255
21:00:00    212
           ... 
16:02:00      1
17:15:00      1
21:03:00      1
03:30:00      1
01:55:00      1
Name: Time_del, Length: 84, dtype: int64

In [24]:
df["Day"].value_counts()

Wednesday    625
Monday       596
Friday       590
Tuesday      589
Thursday     578
Saturday     556
Sunday       544
Ongoing       77
Name: Day, dtype: int64

In [25]:
df.filter(regex='Label_').sum()

Label_Audio                        2696.0
Label_Open                         2882.0
Label_Video                        3766.0
Label_Forum                          19.0
Label_Big Book                     1147.0
Label_Discussion                   1988.0
Label_Speaker                       882.0
Label_Closed                       1113.0
Label_Telephone                    1197.0
Label_Women                         312.0
Label_Loners / Isolationists         16.0
Label_Chat                          204.0
Label_LGBTQIAA+                     224.0
Label_Young People                  102.0
Label_Men                           109.0
Label_Secular                        92.0
Label_Steps / Traditions             80.0
Label_ASL Interpreter                 1.0
Label_Game                           24.0
Label_Blind / Visually Impaired      17.0
Label_Deaf / Hard of Hearing         16.0
Label_Email                          58.0
Label_Professionals                   1.0
Label_Meditation                  

In [26]:
df.count()

Name                               4155
Day                                4155
Time                               4078
Video                              3855
Desc                               4148
Categories                         4155
Email                              3717
Phone                              1597
Time_dt                            4078
Time_del                           4078
Label_Audio                        4155
Label_Open                         4155
Label_Video                        4155
Label_Forum                        4155
Label_Big Book                     4155
Label_Discussion                   4155
Label_Speaker                      4155
Label_Closed                       4155
Label_Telephone                    4155
Label_Women                        4155
Label_Loners / Isolationists       4155
Label_Chat                         4155
Label_LGBTQIAA+                    4155
Label_Young People                 4155
Label_Men                          4155
